- Ziel: wie kann ich einen städtischen Datensatz mit Crowdsourced Zusatzinfos anreichern (OSM + WikiData)
- Jupyter Notebook mit Geopandas + Folium Karte
- Daten
    - OGD/GeoServer: Kunst im Stadtraum (KiöR und KuB)
    - Zeigen: Overpass Query + OSM Wiki (Wie suche ich nach passendes Tags?), taginfo
    - Join mit OSM: `tourism=artwork`
    - Join mit WikiData zu einzelnen Kunstwerken, ihren Künstlern + Foto vom Künstler und Kunstwerk
- OSM
    - OSM Wiki zeigen
    - Overpass Turbo zeigen + Export als GeoJSON
- WikiData
    - Was ist es?
    - Kurze Erklärung zum Datenmodell
    - Wie kann ich Geodaten aus WikiData beziehen?
    - SPARQL-Abfrage für KioR (Bild + Text + Link auf Wikipedia
    - Weitergehende Infos hier: https://www.wikidata.org/wiki/Wikidata:Training
- Ganzer Datensatz zum KioR joinen und dann als GPKG exportieren und z.B. in QGIS öffnen
- Vor/Nachteile, für welche Datensätze eignet sich dieses Vorgehen?
- Quellen und weiterführende Infos zu WikiData + OSM

# Crowdsouring-Daten nutzen

Daten aus OpenStreetMap (OSM) und WikiData zusammen mit städtischen Daten nutzen

In [ ]:
import os
import json
from pprint import pprint

import requests
import folium
import geopandas
import pandas as pd
from IPython.display import HTML, display

import utils

In [ ]:
lv95 = 'EPSG:2056'
wgs84 = 'EPSG:4326'

# Städtische Daten «Kunst im öffentlichen Raum (KiöR)» laden

Im ersten Teil schauen wir uns an, wie wir städtische Daten mit Daten aus OpenStreetMap (OSM) anreichern können.
Dazu laden wir uns zuerst den Datensatz «Kunst im öffentlichen Raum» via WFS.

[Via OGD-Portal](https://data.stadt-zuerich.ch/dataset?q=kunst+im+%C3%B6ffentlichen+raum) finden wir den Datensatz [Kunst im Stadtraum](https://data.stadt-zuerich.ch/dataset/geo_kunst_im_stadtraum).
Von dort via GeoJSON auf das [Geoportal](https://www.stadt-zuerich.ch/geodaten/download/Kunst_im_Stadtraum?format=10009) zur richtigen URL.

In [ ]:
# Variablen setzen
kioer_geojson_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Kunst_im_Stadtraum?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_kioer'
kioer_layer = 'view_kioer'
wfs_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Kunst_im_Stadtraum' 
layers = utils.get_layers_from_wfs(wfs_url)

In [ ]:
# WFS GetFeature Request absetzen
r = requests.get(wfs_url, params={
    'service': 'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typename': kioer_layer,
    'outputFormat': 'GeoJSON'
})
kioer_geo = r.json()
kioer_geo

# Daten aus OpenStreetMap laden

Wir suchen thematisch passende Daten aus OpenStreetMap.
Um diese zu finden, lohnt es sich das [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/Main_Page) zu durchsuchen und passende Tags zu finden.

## OSM-Daten laden

Daten von OpenStreetMap (OSM) können u.a. via Overpass API geladen werden.
Overpass hat eine eigene Abfragesprache ([Overpass QL](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL)), mit der Objekte (Nodes, Ways, Relations) abgefragt werden können.

![OpenStreetMap Datenmodell](https://user-images.githubusercontent.com/538415/136281860-2af43355-bcb9-47fd-8a34-1c86cf290dbc.png)
[Bildquelle: &copy; Itinero](https://docs.itinero.tech/docs/osmsharp/osm.html)

[Query in Overpass Turbo ausführen](https://osm.li/UfK)

In [ ]:
# Overpass Query absetzen
artwork_zh = """
/*
Alle Kunstwerke (tourism=artwork) in der Stadt Zürich
*/
[out:json];
area["name"="Zürich"]["wikipedia"="de:Zürich"]->.perimeter; 
(
  nwr[tourism=artwork](area.perimeter);
);
out center;
"""
result = utils.overpass_query(artwork_zh)
result

## Karte mit den Resultaten

In [ ]:
# Basiskarte
m = utils.base_map()

# KiöR-Daten hinzufügen
kioer_layer = folium.FeatureGroup(name='KiöR', show=True)
utils.style_layer(kioer_geo, kioer_layer, icon_color='#031cff', icon='certificate', prefix='fa')
kioer_layer.add_to(m)

# OSM-Daten hinzufügen
osm_layer = folium.FeatureGroup(name='OSM: tourism=artwork', show=True)
utils.style_layer(result, osm_layer, icon_color='#ff033e', icon='fire', prefix='fa')
osm_layer.add_to(m)

# Add controls for layers
folium.LayerControl().add_to(m)
m

## Spatial Join der zwei Quellen

In [ ]:
kioer_df = geopandas.GeoDataFrame.from_features(kioer_geo, crs=wgs84)
kioer_df.head()

In [ ]:
osm_df = geopandas.GeoDataFrame.from_features(result, crs=wgs84)
osm_df.head()

In [ ]:
# drop all elements with empty geometry
kior_buf = kioer_df.dropna(subset=['geometry']).reset_index(drop=True)
osm_buf = osm_df.dropna(subset=['geometry']).reset_index(drop=True)

# CRS zu LV95 re-projezieren
kior_buf = kior_buf.to_crs(lv95) # convert to plannar coordinate system
osm_buf = osm_buf.to_crs(lv95) # convert to plannar coordinate system

# Buffer um die Geometrien hinzufügen (10 Meter)
kior_buf['geometry'] = kior_buf.geometry.buffer(10) 
osm_buf['geometry'] = osm_buf.geometry.buffer(10)

In [ ]:
# Basiskarte 
bm = utils.base_map()

folium.features.GeoJson(
    kior_buf.to_crs(wgs84).to_json(),
    tooltip=folium.features.GeoJsonTooltip(
        fields=['titel', 'autoren', 'datierung', 'material'],
        aliases=['Titel:', 'Künstler:', 'Datierung:', 'Material:'],    
    )
).add_to(bm)

folium.features.GeoJson(
    osm_buf.to_crs(wgs84).to_json(),
    style_function=lambda x: {'fillColor': '#FF0000', 'color': '#FF0000'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'artist_name', 'wikidata'],
        aliases=['Titel:', 'Künstler:', "Wikidata:"],                      
    )
).add_to(bm)
bm

In [ ]:
# spatial join über die beiden Geometrien
sjoin_kunst = geopandas.sjoin(kior_buf, osm_buf, how='left', predicate='intersects', lsuffix='kior', rsuffix='osm').reset_index()

# Wie zurück zu WGS84
sjoin_kunst = sjoin_kunst.to_crs(wgs84)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sjoin_kunst[['titel', 'name', 'autoren', 'artist_name', 'material_kior', 'material_osm', 'datierung']])

In [ ]:
folium.features.GeoJson(
    sjoin_kunst.to_json(),
    style_function=lambda x: {'fillColor': '#09bd63', 'color': '#09bd63'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'artist_name', 'wikidata'],
        aliases=['Titel:', 'Künstler:', "Wikidata:"],                      
    )
).add_to(bm)
bm

In [ ]:
# OSM-Einträge ohne Match
osm_no_match = osm_df[(~osm_df.id.isin(sjoin_kunst.id))].reset_index()
osm_no_match.head()

In [ ]:
osm_no_match.to_file(os.path.join('..', 'kunst_package.gpkg'), layer='osm_no_match', driver="GPKG")
sjoin_kunst.to_file(os.path.join('..', 'kunst_package.gpkg'), layer='kioer_osm_match', driver="GPKG")

In [ ]:
# Das GeoPackage lässt sich z.B. in QGIS öffnen

# Daten aus WikiData laden

## Daten via WikiData-Verweise von OSM holen

[OpenStreetMap Node «Heureka»](https://www.openstreetmap.org/node/268472148)


In [ ]:
# WikiData Verweise von OpenStreetMap
osm_kunst_wd = osm_df[(~pd.isna(osm_df.wikidata))] # alle Einträge, bei denen "wikidata" nicht leer ist
osm_kunst_wd = osm_kunst_wd[['geometry', 'type', 'id', 'artist_name', 'artist:wikidata', 'name', 'wikidata']].reset_index(drop=True)
osm_kunst_wd.head()

### Beispiel-Abfrage nach WikiData Item

[Wikidata-Item «Heureka»](https://www.wikidata.org/wiki/Q1378316)

In [ ]:
wikidata_item = utils.wikidata_item(osm_kunst_wd['wikidata'].iloc[1])
pprint(wikidata_item)

In [ ]:
category = wikidata_item['sitelinks']['commonswiki']['title']
urls = utils.images_from_commons_category(category, image_size=500)
urls

In [ ]:
display(HTML(''.join([utils.img_html(url) for url in urls])))

## Daten via SPARQL aus WikiData beziehen

Um Daten aus WikiData zu laden, können Abfragen mit SPARQL gemacht werden. SPARQL ist eine SQL-ähnliche Abfragesprache für Linked Data.

Die Idee von Linked Data ist es, einen Informations-Graphen in Form von sogenannten «Triples» abzubilden:

![Linked Data Triple](assets/images/download.webp)

[Bildquelle: &copy; WordLift](https://wordlift.io/blog/en/entity/linked-data/)

**Triple = Subjekt (z.B. Marie) -> Prädikat (z.B. birthPlace) -> Objekt (z.B. Italy)**

SPARQL-Query

[Link zum Wikidata Query Service](https://w.wiki/4BJ8)

In [ ]:
# direkte SPARQL-Query auf WikiData
wd_kunstwerke_query = """
SELECT DISTINCT ?artwork ?artworkLabel ?creator ?creatorLabel ?creatorBirthday ?createDateOfDeath ?geo ?lat ?lon
WHERE
{
  ?artwork wdt:P136 wd:Q557141 .           # Genre "Kunst im öffentlichen Raum"
  ?artwork wdt:P131 wd:Q72 .               # liegt in Zürich
  OPTIONAL {
    ?artwork wdt:P170 ?creator .           # Urheber des Werks
    ?creator wdt:P569 ?creatorBirthday .
    ?creator wdt:P570 ?createDateOfDeath .
  }   
  OPTIONAL {
    ?artwork wdt:P625 ?geo .               # Koordinaten für Kartenansicht
    ?artwork p:P625/psv:P625 ?coord .
    ?coord wikibase:geoLatitude ?lat .
    ?coord wikibase:geoLongitude ?lon .
  }       
 
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de,en"
  }
}
ORDER BY ?artworkLabel
"""
result = utils.wikidata_query(wd_kunstwerke_query)
result

In [ ]:
flat_result = [{k: r[k]['value'] for k,v in r.items()} for r in result]
wikidata_df = pd.DataFrame(flat_result)
wikidata_df

# Anhang und Tutorials

## OpenStreetMap (OSM)


## WikiData

* Weitergehende Infos hier: https://www.wikidata.org/wiki/Wikidata:Training